## Data Processing

In [32]:
import os
import boto3
import numpy as np

import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import date_format
from pyspark.sql.functions import *
from pyspark.ml import Pipeline

In [2]:
sc = pyspark.SparkContext.getOrCreate()
ss = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
bucket_name = 'msds697jonross.and.friends' # Add your bucket name
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name) 

In [4]:
file_name_fire = 'sffd.csv' # select file
obj_fire = bucket.Object(key=file_name_fire) # S3 uses key-value structure where key is your file name
file_content_fire = obj_fire.get()["Body"].read().decode("utf-8") # Read the Body which is the contents of the file.

In [5]:
file_name_goo = 'google_data.csv' # select file
obj_goo = bucket.Object(key=file_name_goo) # S3 uses key-value structure where key is your file name
file_content_goo = obj_goo.get()["Body"].read().decode("utf-8") # Read the Body which is the contents of the file.

In [6]:
# number of rows (subract header and empty line at end)
rows_fire = file_content_fire.split('\n')
len(rows_fire)-2

4557045

In [7]:
# number of rows (subract empty line at end)
rows_goo = file_content_goo.split('\n')
len(rows_goo)-1

32074

In [8]:
# number of  columns
column_names_fire = rows_fire[0].split(',')
n_cols = sc.broadcast(len(column_names_fire))
n_cols.value

14

In [9]:
print('\n'.join(x for x in column_names_fire))

call_type
received_timestamp
entry_timestamp
dispatch_timestamp
response_timestamp
on_scene_timestamp
transport_timestamp
hospital_timestamp
call_final_disposition
available_timestamp
address
zipcode_of_incident
battalion
station_area


In [14]:
# randomly sample rows
np.random.seed(1)
sz=100000
samples_fire = np.random.choice(rows_fire[1:], size=sz, replace=False)
samples_fire[:2]

array(['Medical Incident,2012-06-20 13:12:20+00:00,2012-06-20 13:12:53+00:00,2012-06-20 13:13:54+00:00,2012-06-20 13:14:19+00:00,2012-06-20 13:19:11+00:00,,2012-06-20 13:43:07+00:00,Code 2 Transport,2012-06-20 14:11:47+00:00,600 Block of CASTRO ST,94114,B06,06',
       'Structure Fire,2010-01-04 11:23:56+00:00,2010-01-04 11:24:22+00:00,2010-01-04 11:24:47+00:00,2010-01-04 11:46:53+00:00,,,,Other,2010-01-04 12:01:03+00:00,3400 Block of 24TH ST,94110,B06,11'],
      dtype='<U315')

In [15]:
stations = [f"0{i}" for i in range(1,10)] +\
    [str(i) for i in list(range(10,27))] +\
    ['28','29'] +\
    [str(i) for i in list(range(31,45))] +\
    ['48','49','51']
stations = sc.broadcast(stations)
# stations.value

In [16]:
def filter_fire(x):
    if len(x.split(',')) == n_cols.value:
        return x.split(',')[13] in stations.value

def map_fire(x):
    return ((x.split(',')[13],x.split(',')[10]), x.split(',')[:2]+[x.split(',')[5]]+[x.split(',')[11]])

def map_google(x):
    s = x.split(',')
    try:
        if int(s[0]) < 10:
            s[0] = '0' + s[0]
    except:
        pass
    return (tuple(s[:2]), s[2:])

def map_joined(x):
    try:
        return list(x[0])+x[1][0]+[float(y) for y in x[1][1]]
    except:
        return list(x[0])+x[1][0]+[None,None]

In [17]:
rdd_fire = sc.parallelize(list(samples_fire))\
    .filter(filter_fire)\
    .map(map_fire)

In [18]:
rdd_google = sc.parallelize(rows_goo)\
    .map(map_google)

In [19]:
rdd_fire.take(2)

[(('06', '600 Block of CASTRO ST'),
  ['Medical Incident',
   '2012-06-20 13:12:20+00:00',
   '2012-06-20 13:19:11+00:00',
   '94114']),
 (('11', '3400 Block of 24TH ST'),
  ['Structure Fire', '2010-01-04 11:23:56+00:00', '', '94110'])]

In [20]:
rdd_google.take(2)

[(('01', '0 Block of 0NB OCTAVIA OF'), ['3724', '12.883333333333333']),
 (('01', '0 Block of 101 NB OCTAVIA OF'), ['4290', '9.416666666666666'])]

In [21]:
joined = rdd_fire.leftOuterJoin(rdd_google)

In [22]:
joined.take(2)

[(('03', '500 Block of ELLIS ST'),
  (['Structure Fire',
    '2004-06-17 18:52:58+00:00',
    '2004-06-17 19:00:36+00:00',
    '94109'],
   ['681', '2.816666666666667'])),
 (('03', '500 Block of ELLIS ST'),
  (['Alarms',
    '2011-03-04 04:01:43+00:00',
    '2011-03-04 04:09:13+00:00',
    '94109'],
   ['681', '2.816666666666667']))]

In [23]:
rdd = joined.map(map_joined)

In [24]:
rdd.take(2)

[['03',
  '500 Block of ELLIS ST',
  'Structure Fire',
  '2004-06-17 18:52:58+00:00',
  '2004-06-17 19:00:36+00:00',
  '94109',
  681.0,
  2.816666666666667],
 ['03',
  '500 Block of ELLIS ST',
  'Alarms',
  '2011-03-04 04:01:43+00:00',
  '2011-03-04 04:09:13+00:00',
  '94109',
  681.0,
  2.816666666666667]]

In [25]:
# number of rows removed
sz - rdd.count()

1475

In [58]:
schema = StructType([StructField("station_area", StringType(), False),
                     StructField("address", StringType(), False),
                     StructField("call_type", StringType(), False),
                     StructField("received_timestamp", StringType(), False),
                     StructField("on_scene_timestamp", StringType(), False),
                     StructField("zipcode_of_incident", StringType(), False),
                     StructField("distance", DoubleType(), True),
                     StructField("duration", DoubleType(), True)
                     ])

In [59]:
df = ss.createDataFrame(rdd, schema) # .cache()

In [60]:
df.printSchema()

root
 |-- station_area: string (nullable = false)
 |-- address: string (nullable = false)
 |-- call_type: string (nullable = false)
 |-- received_timestamp: string (nullable = false)
 |-- on_scene_timestamp: string (nullable = false)
 |-- zipcode_of_incident: string (nullable = false)
 |-- distance: double (nullable = true)
 |-- duration: double (nullable = true)



In [61]:
# convert to timestamps
my_rows = ['received_timestamp',
          'on_scene_timestamp']

df_w_time = df
for row in my_rows:
    df_w_time = df_w_time.withColumn(row, to_timestamp(df[row], format = 'yyyy-MM-dd HH:mm:ss+00:00'))

In [62]:
df_w_time = df_w_time.withColumn("received_hour",
                                 hour("received_timestamp")) \
            .withColumn("day_of_week",
                        date_format('received_timestamp', 'u').alias('dow_number'))
# df_w_time.show(10)

## Create More Interesting DataFrame

In [295]:
data_full = df_w_time.select('station_area',
                 'address',
                 'call_type',
                 'received_timestamp',
                 'on_scene_timestamp',
                 'zipcode_of_incident',
                 'distance',
                 'duration',
                 'received_hour',
                 'day_of_week')\
    .withColumn("distance", 
                col('distance') / 1609.34 )\
    .withColumn("label", 
                (unix_timestamp('on_scene_timestamp') - unix_timestamp('received_timestamp')) / 60 )\
    .withColumn("speed", 
                col('distance') / col('duration') * 60 )\
    .withColumn("label_ratio", 
                col('label') / col('duration') )\
    .orderBy('received_timestamp', ascending=[0])\
    .na.drop(subset=["label"])\
    .where('label >= 0')\
    .select('station_area',
        'call_type',
        'zipcode_of_incident',
        'distance',
        'duration',
        'speed',
        'label_ratio',
        'received_hour',
        'day_of_week',
        'label')

# data_1 is the largest dataset
# has all columns and ony removes missing google data
data_1 = data_full.na.drop(subset=["distance", "duration"]) # (removes about 50 rows)

In [296]:
data_1.printSchema()

root
 |-- station_area: string (nullable = false)
 |-- call_type: string (nullable = false)
 |-- zipcode_of_incident: string (nullable = false)
 |-- distance: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- label_ratio: double (nullable = true)
 |-- received_hour: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- label: double (nullable = true)



- `label`: response time
- `duration`: google api estimate of driving time from station to address (in minutes)
- `distance`: google api estimate of driving distance from station to address
- `speed`: distance / duration * 60 (estimated average speed in mph)
- `label_ratio`: label / duration. **Using this variable for eda**. The idea is to try to find the observations where distance and duration would underestimate most severely.

In [297]:
data_1.approxQuantile("distance", [.0001,.001,.5,.9,.97,.98,.99,.999], 0)

[0.0,
 0.01553431841624517,
 0.5710415449811724,
 1.2334248822498666,
 1.880895273838965,
 2.403469745361453,
 5.0772366311655714,
 10.992083711335082]

In [298]:
# removing rows where distance (in miles) is too large or too small
data_2 = data_1.where('distance between .01 and 2') 
print(f'removed {data_1.count() - data_2.count()} rows')

removed 2063 rows


In [299]:
# check to see that speeds look reasonable (mph)
data_2.approxQuantile("speed", [.001,.01,.02,.03,.1,.5,.9,.96,.97,.98,.99,.999], 0)

[5.1621735044753185,
 6.018438792122414,
 6.363339442895303,
 6.53586920985618,
 7.339965451675843,
 9.760370774380068,
 13.378074322312331,
 15.084683683777293,
 15.638797017982748,
 16.75797735497192,
 18.40977432170738,
 21.29874549624261]

In [300]:
data_2.approxQuantile("label", [.001,.01,.02,.03,.1,.5,.9,.96,.97,.98,.99,.999], 0)

[0.0,
 1.9,
 2.5166666666666666,
 2.816666666666667,
 3.816666666666667,
 6.566666666666666,
 14.566666666666666,
 21.2,
 23.5,
 26.783333333333335,
 33.35,
 77.85]

In [301]:
# removing rows where response time (in minutes) is too large or too small
data_3 = data_2.where('label between 1 and 30') 
print(f'removed {data_2.count() - data_3.count()} rows')

removed 1404 rows


In [302]:
data_3.approxQuantile("label_ratio", [.001,.01,.1,.5,.9,.96,.97,.98,.99,.999], 0)

[0.2263083451202263,
 0.3801801801801802,
 0.8526315789473685,
 2.0104166666666665,
 5.546218487394958,
 8.72340425531915,
 10.022727272727273,
 12.333333333333334,
 17.705882352941178,
 68.00000000000001]

In [303]:
# removing rows where response time is too much larger than duration
data_4 = data_3.where('label_ratio < 20') 
print(f'removed {data_3.count() - data_4.count()} rows')

removed 595 rows


In [304]:
data_cleaned = data_4\
    .select('station_area','call_type','zipcode_of_incident',
            'distance','duration','received_hour','day_of_week','label')
data_cleaned.select('station_area','call_type','zipcode_of_incident').show()
data_cleaned.select('distance','duration','received_hour','day_of_week','label').show()

+------------+-----------------+-------------------+
|station_area|        call_type|zipcode_of_incident|
+------------+-----------------+-------------------+
|          11|           Alarms|              94110|
|          31| Medical Incident|              94118|
|          11|           Alarms|              94131|
|          07|   Structure Fire|              94110|
|          07| Medical Incident|              94110|
|          29|Traffic Collision|              94103|
|          36| Medical Incident|              94102|
|          05|Traffic Collision|              94102|
|          36| Medical Incident|              94103|
|          06| Medical Incident|              94114|
|          36| Medical Incident|              94103|
|          01|           Alarms|              94103|
|          36| Medical Incident|              94102|
|          17| Medical Incident|              94124|
|          31|           Alarms|              94121|
|          05| Medical Incident|              

## Machine Learning

In [316]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor

In [306]:
def remove_empty_strings(df):
    return df.replace('','unknown')
# dfnonas = remove_empty_strings(small_df)

In [307]:
#converting strings to numeric values
def indexStringColumns(df, cols):
    #variable newdf will be updated several times
    newdf = df
    
    for c in cols:
        #For each given colum, fits StringIndexerModel.
        si = StringIndexer(inputCol=c, outputCol=c+"-num")
        sm = si.fit(newdf)
        #Creates a DataFame by putting the transformed values in the new colum with suffix "-num" 
        #and then drops the original columns.
        #and drop the "-num" suffix. 
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf

# dfnumeric = indexStringColumns(dfnonas, ["call_type", "zipcode_of_incident", "station_area"])
# dfnumeric = indexStringColumns(dfnonas, [])

In [308]:
def oneHotEncodeColumns(df, cols):
    newdf = df
    for c in cols:
        #For each given colum, create OneHotEncoder. 
        #dropLast : Whether to drop the last category in the encoded vector (default: true)
        onehotenc = OneHotEncoder(inputCol=c, outputCol=c+"-onehot", dropLast=False)
        #Creates a DataFame by putting the transformed values in the new colum with suffix "-onehot" 
        #and then drops the original columns.
        #and drop the "-onehot" suffix. 
        newdf = onehotenc.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-onehot", c)
    return newdf

# dfhot = oneHotEncodeColumns(dfnumeric, ["call_type", "zipcode_of_incident", "station_area"])
# dfhot = oneHotEncodeColumns(dfnumeric, [])

In [309]:
def process(df, cat_cols, hot_cols):
    dfnonas = remove_empty_strings(df)
    dfnumeric = indexStringColumns(dfnonas, cat_cols)
    dfhot = oneHotEncodeColumns(dfnumeric, hot_cols)
    return dfhot

### Choose data

In [339]:
# one or the other
processed_data = process(data_cleaned.select('distance','duration','label'),
                         cat_cols=[],
                         hot_cols=[])

# processed_data = process(data_cleaned,
#                          cat_cols=["call_type", "zipcode_of_incident", "station_area", "received_hour", "day_of_week"],
#                          hot_cols=["call_type", "zipcode_of_incident", "station_area", "received_hour", "day_of_week"])

In [340]:
# Take approximately top ~10% of rows convert them to dataframe 
# Also provide the schema also to avoid errors
test = ss.createDataFrame(processed_data.head(8000), processed_data.schema)

#Take the rest of the rows
train = processed_data.subtract(test)

In [341]:
print(train.count(), test.count())

61029 8000


In [342]:
# make pipelines
va = VectorAssembler(outputCol="features",
                     inputCols=[x for x in processed_data.columns if x != 'label'])

rf = RandomForestRegressor()
dt = DecisionTreeRegressor()
lr = LinearRegression()
lr_ElasticNet = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

pipeline_lr = Pipeline(stages=[va,lr])
pipeline_lr_ElasticNet = Pipeline(stages=[va,lr_ElasticNet])
pipeline_dt = Pipeline(stages=[va,dt])
pipeline_rf = Pipeline(stages=[va,rf])

In [343]:
# fit models
lr_fitted = pipeline_lr.fit(train)
lr_e_fitted = pipeline_lr_ElasticNet.fit(train)
dt_fitted = pipeline_dt.fit(train)
rf_fitted = pipeline_rf.fit(train)

## Results - careful about re-running cells
In each section, first cell is results on all columns and second cell uses just `distance` and `duration`. We see that the metrics are consistently better when we have use all of the columns and not just these two.

In [344]:
# Evaluate model results
def evaluate_regression(fitted_model, test_set):
    test_pred = fitted_model.transform(test_set)
    r2_ev = RegressionEvaluator(metricName='r2')
    rmse_ev = RegressionEvaluator(metricName='rmse')
    mae_ev = RegressionEvaluator(metricName='mae')
    r2 = r2_ev.evaluate(test_pred)
    rmse = rmse_ev.evaluate(test_pred)
    mae = mae_ev.evaluate(test_pred)
    return (r2,rmse,mae)

In [345]:
# just distance and duration
ev_lr = evaluate_regression(fitted_model=lr_fitted, test_set=test)
print(f"Linear Regression:\nR^2:  {ev_lr[0]}\nRMSE: {ev_lr[1]}\nMAE:  {ev_lr[2]}")
print('\n')
ev_e = evaluate_regression(fitted_model=lr_e_fitted, test_set=test)
print(f"Elastic Net Regression:\nR^2:  {ev_e[0]}\nRMSE: {ev_e[1]}\nMAE:  {ev_e[2]}")
print('\n')
ev_dt = evaluate_regression(fitted_model=dt_fitted, test_set=test)
print(f"Decision Tree:\nR^2:  {ev_dt[0]}\nRMSE: {ev_dt[1]}\nMAE:  {ev_dt[2]}")
print('\n')
ev_rf = evaluate_regression(fitted_model=rf_fitted, test_set=test)
print(f"Random Forest\nR^2:  {ev_rf[0]}\nRMSE: {ev_rf[1]}\nMAE:  {ev_rf[2]}")

Linear Regression:
R^2:  -0.022815405417011325
RMSE: 4.96772438969419
MAE:  3.4244997440878224


Elastic Net Regression:
R^2:  -0.02608933445342676
RMSE: 4.975668630164337
MAE:  3.4403794737203923


Decision Tree:
R^2:  -0.01782338654641391
RMSE: 4.955586663423365
MAE:  3.4258079945645683


Random Forest
R^2:  -0.01684640014176164
RMSE: 4.953207712791327
MAE:  3.4238245095136017


In [336]:
# all predictors
ev_lr = evaluate_regression(fitted_model=lr_fitted, test_set=test)
print(f"Linear Regression:\nR^2:  {ev_lr[0]}\nRMSE: {ev_lr[1]}\nMAE:  {ev_lr[2]}")
print('\n')
ev_e = evaluate_regression(fitted_model=lr_e_fitted, test_set=test)
print(f"Elastic Net Regression:\nR^2:  {ev_e[0]}\nRMSE: {ev_e[1]}\nMAE:  {ev_e[2]}")
print('\n')
ev_dt = evaluate_regression(fitted_model=dt_fitted, test_set=test)
print(f"Decision Tree:\nR^2:  {ev_dt[0]}\nRMSE: {ev_dt[1]}\nMAE:  {ev_dt[2]}")
print('\n')
ev_rf = evaluate_regression(fitted_model=rf_fitted, test_set=test)
print(f"Random Forest\nR^2:  {ev_rf[0]}\nRMSE: {ev_rf[1]}\nMAE:  {ev_rf[2]}")

Linear Regression:
R^2:  0.040529356313042886
RMSE: 4.8114359463015095
MAE:  3.296128921495305


Elastic Net Regression:
R^2:  0.0056151653693420345
RMSE: 4.898195431392101
MAE:  3.3539429031301595


Decision Tree:
R^2:  0.03007135145173201
RMSE: 4.837586641684353
MAE:  3.3317944451983403


Random Forest
R^2:  0.027825524400997392
RMSE: 4.843184012803788
MAE:  3.3224205420135933


It looks like there is quite a bit of overfitting going on here. That isn't too surprising since there wasn't too much preprocessing done other than outlier removal. Some of the categorical features have a lot of categories but our target variable may not change much across those categories which could also be a problem. Hyper-parameters weren't chosen very carefully. Right now, regularized regression doesn't really help the linear model and ensembling doesn't really help the decision tree, but maybe better hyper-parameters would change that.